<a href="https://colab.research.google.com/github/BrainStormYourWayIn/sierra_doc/blob/main/bookshelf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sierra
!pip install flask_ngrok

import os

mypath = 'templates/'
if not os.path.isdir(mypath):
   os.makedirs(mypath)

with open('templates/form.html', 'w') as f:
    f.write('''
<!DOCTYPE html>
<html lang="en">
 <meta charset="utf-8"/>
 <head>
  <title>
   test
  </title>
  <link href="/static/style.css" rel="stylesheet"/>
 </head>
 <body>
  <form action="/" method="post">
   <label for="bookname">
    Book Name:
    <input id="bookname" name="bookname" placeholder="bookname" type="text"/>
   </label>
   <label for="authorname">
    Author's Name:
    <input id="authorname" name="authname" placeholder="authorname" type="text"/>
   </label>
   <button type="submit">
    Search
   </button>
  </form>
  <p>
      Try searching for 'Automate the Boring Stuff with Python'
  </p>
  <p>
      Enter only one of the two fields. Entering both results in ignoring the author's name field.
  </p>
 </body>
</html>
''')
# http://libgen.rs/search.php?req=sri+m&open=0&res=25&view=simple&phrase=1&column=author  --> author

# http://libgen.rs/search.php?req=sri+m&lg_topic=libgen&open=0&view=simple&res=25&phrase=1&column=title  --> title

In [4]:
from flask import Flask, request, render_template, url_for, redirect
from flask_ngrok import run_with_ngrok
from bs4 import BeautifulSoup
import requests
from sierra import *

app = Flask(__name__)
run_with_ngrok(app)

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:84.0) Gecko/20100101 Firefox/84.0",
}

@app.route('/results')
def something_else():
    

    b_name = request.args.get('book_name')
    a_name = request.args.get('auth_name')

    b_name = b_name.replace(' ', '+')
    a_name = a_name.replace(' ', '+')

    if b_name and not a_name or b_name and a_name:

        title('Search Results', templating=True)
        openBody()

        libgen = requests.get(f'http://libgen.rs/search.php?req={b_name}&open=0&res=25&view=simple&phrase=1&column=title').text
        libgen_soup = BeautifulSoup(libgen, 'html.parser').find_all("a", title='Gen.lib.rus.ec', href=True)

        for link in libgen_soup:
            # print(link['href'])
            link_libgen = requests.get(url=link['href'], headers=headers).text
            link_libgen_soup = BeautifulSoup(link_libgen, 'html.parser').find_all("h1")
            for item in link_libgen_soup:
                for thing in item.contents:
                    writeWA(f"<a href='{link['href']}' tagret='_blank'>{thing}</a><br/>")

        writeWA("<br><br>")

        page = requests.get(f'https://html.duckduckgo.com/html/?q={b_name}', headers=headers).text
        soup = BeautifulSoup(page, 'html.parser').find_all("a", class_="result__a")

        for link in soup:
            writeWA(f"<a href='{link['href']}' target='_blank'>{link.text}</a><br/>")

        writeWA("<br><br>")

        writeWA(f"<a href='https://www.google.com/search?q=intitle.index+of+pdf+{b_name}'>Index search</a>")
    

        autoPrettify()
        write_to_template('search_results')
        return render_template('search_results')

    if a_name and not b_name:

        title('Search Results', templating=True)
        openBody()

        libgen = requests.get(f'http://libgen.rs/search.php?req={a_name}&open=0&res=25&view=simple&phrase=1&column=author').text
        libgen_soup = BeautifulSoup(libgen, 'html.parser').find_all("a", title='Gen.lib.rus.ec', href=True)

        for link in libgen_soup:
            # print(link['href'])
            link_libgen = requests.get(url=link['href'], headers=headers).text
            link_libgen_soup = BeautifulSoup(link_libgen, 'html.parser').find_all("h1")
            for item in link_libgen_soup:
                for thing in item.contents:
                    writeWA(f"<a href='{link['href']}' tagret='_blank'>{thing}</a><br/>")

        writeWA("<br><br>")

        page = requests.get(f"https://html.duckduckgo.com/html/?q={a_name}'s+books", headers=headers).text
        soup = BeautifulSoup(page, 'html.parser').find_all("a", class_="result__a")

        for link in soup:
            writeWA(f"<a href='{link['href']}' target='_blank'>{link.text}</a><br/>")

        writeWA("<br><br>")

        writeWA(f"<a href='https://www.google.com/search?q=intitle.index+of+pdf+{a_name}'>Index search</a>")
    

        autoPrettify()
        write_to_template('search_results')

        return render_template('search_results')


    if not a_name and not b_name:
        return "<h2>Enter either an author, or a book's name</h2>"


    # return render_template('search_results')               



@app.route('/', methods =["GET", "POST"])
def gfg():
    if request.method == "POST":

        book_name = request.form['bookname']
        author_name = request.form['authname']

        return redirect(url_for('something_else', book_name=book_name, auth_name=author_name))

    return render_template('form.html')

# from google.colab.output import eval_js
# print(eval_js("google.colab.kernel.proxyPort(8000)"))

if __name__ == '__main__':
    # app.run(debug=False, port=8000)
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c50fa3148e63.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Jul/2021 06:59:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 06:59:10] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [15/Jul/2021 06:59:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Jul/2021 06:59:39] "POST / HTTP/1.1" 302 -
<class 'str'>:186: UserWarning: Auto prettifying also involves auto closing unclosed HTML tags which may not be accurate 
if not used after development is complete. 
Use after all development for best results. See "bs4 auto closing tags" for more info.
127.0.0.1 - - [15/Jul/2021 06:59:54] "GET /results?book_name=automate+the+boring+stuff+with+python&auth_name= HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 06:59:55] "GET /static/style.css HTTP/1.1" 404 -
